# Planetary Nebula <a class="tocSkip">
    
This notebook is used to test and showcase the results of my first project. I use spectroscopic data from the [Multi Unit Spectroscopic Explorer](https://www.eso.org/sci/facilities/develop/instruments/muse.html) (MUSE) that has been observed as part of the [PHANGS](https://sites.google.com/view/phangs/home) collaboration.
    
I will use a set of line maps of emission lines to identify Planetary Nebula in the data an measure their brightness. This can then be used to fit an empiric relation and hence measure the distance to the galaxy.
    
This notebook is used for developement. Final code is moved to the `pymuse` packge in the `src` folder. Any production scripts reside in the `scripts` folder.

## Preparation
 
### Load Basic Packages
    
First we load a bunch of common packages that are used across the project. More specific packages that are only used in one section are loaded later to make it clear where they belong to (this also applies to all custom moduls that were written for this project).

In [226]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

# some basic packages
import os                 # filesystem related stuff
import json
from pathlib import Path  # use instead of os.path and glob
import sys                # mostly replaced by pathlib

import errno      # more detailed error messages
import warnings   # handles warnings
import logging    # use logging instead of print

from collections import OrderedDict  

# packages for scientific computing
import numpy as np
import scipy as sp

# packages for creating plots and figures
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# special functions for astronomy 
from astropy.table import Table  # useful datastructure
from astropy.table import vstack # combine multiple tables

from astropy.io import fits      # open fits files
from astropy.io import ascii     # handle normal files

from astropy.wcs import WCS               # handle coordinates
from astropy.coordinates import SkyCoord  # convert pixel to sky coordinates

from astropy.stats import sigma_clipped_stats  # calcualte statistics of images

import astropy.units as u        # handle units

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


we use the `logging` module to handle informations and warnings (this does not always work as expected in jupyter notebooks).

In [ ]:
logging.basicConfig(stream=sys.stdout,
                    #format='(levelname)s %(name)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

### Read in data

this uses the `ReadLineMaps` class from the `pymuse.io` module. To use it, we first need to specify the path to the data folder

In [ ]:
from pymuse.io import ReadLineMaps

# first we need to specify the path to the raw data
data_raw = Path('d:\downloads\MUSEDAP')
basedir = Path('..')

with open(basedir / 'data' / 'interim' / 'parameters.json') as json_file:
    parameters = json.load(json_file)

# list all files in the specified directory
galaxies = [x.name for x in data_raw.iterdir() if x.is_dir()]
print(', '.join(map(str,galaxies)))

# read in the data we will be working with and print some information
galaxy = ReadLineMaps(data_raw / 'NGC628')
setattr(galaxy,'mu',parameters[galaxy.name]['mu'])
setattr(galaxy,'alpha',parameters[galaxy.name]['power_index'])

#print('\n' + str(NGC628))

In [ ]:
filename = basedir / 'data' / 'raw' / 'phangs_sample_table_v1p4.fits'
with fits.open(filename) as hdul:
    sample_table = Table(hdul[1].data)

### Reference files

To test the newly written routines we compare our results to those from Kreckel et al. (2017)

**Requires** (both already loaded with standard packages)
 * `astropy.io.ascii`
 * `astropy.coordinates.SkyCoord`
 
**Returns**
 * `pn_kreckel` table with all PNe detected by Kreckel et al. (2017)
 * `pn_bright` only PNe that are brighter than the completeness limit
 * `pn_hermann` only PNe that were also detected by Hermann et al. 2008

In [ ]:
pn_kreckel = ascii.read(basedir / 'data' / 'external' / 'kreckel_pn_2017.txt')

def string_to_ra(string):
    '''convert coordinates from Kreckel et al. (2017) to astropy
    
    the right ascension in the paper is given as 
    "01:36:42.212" but astropy requires "01h36m42.212s".
    This function replaces the ":" with the appropriate character.
    '''
    return string.replace(':','h',1).replace(':','m') + 's'

def string_to_dec(string):
    '''convert coordinates from Kreckel et al. (2017) to astropy
    
    the declination in the paper is given as "01:36:42.212" 
    but astropy requires "01d36m42.212s".
    This function replaces the ":" with the appropriate character.
    '''
    return string.replace(':','d',1).replace(':','m') + 's'

# convert string to astronomical coordinates
pn_kreckel['RA'] = list(map(string_to_ra,pn_kreckel['RA']))
pn_kreckel['DEC'] = list(map(string_to_dec,pn_kreckel['DEC']))
pn_kreckel['SkyCoord'] = SkyCoord(pn_kreckel['RA'],pn_kreckel['DEC'])

# select some subsets (PN from Hermann et al. 2008 or bright sources only)
pn_herrmann = pn_kreckel[[True if i.endswith('a') else False for i in pn_kreckel['ID']]]
pn_bright = pn_kreckel[pn_kreckel['mOIII']<27]

## Source Detection

There are two different approaches to identifying sources in an image. The first utilizes PSF fitting and uses implementations from astropy. The other uses the external `SExtractor` package which detects peaks and classifies them with a neural network.

### Based on IRAFStarFinder or DAPStarFinder

The sources we are searching for are unresolved. However due to seeing, they will be smeared out. This PSF has the form of a Gaussian (or Moffat). The subsequent algorithms use this and try to fit a theoretical curve to the observed peaks in the image. If the fit aggrees within some threshold, it reports the peak as a source. The advantage is that for crowded fields, the algorithm will try to fit an individual function to each peak and thus enable us correctly identfiy objects that are closeby.

The following function is based on this tutorial 

https://photutils.readthedocs.io/en/stable/detection.html

https://photutils.readthedocs.io/en/stable/api/photutils.detection.DAOStarFinder.html#photutils.detection.DAOStarFinder

**Requires**
 * A `photutils` starfinder. This can be either `DAOStarFinder` or `IRAFStarFinder`
 * `detect_unresolved_sources`
 
**Returns**
 * `sources` a table with the position of all identified sources

In [ ]:
from photutils import DAOStarFinder            # DAOFIND routine to detect sources
from photutils import IRAFStarFinder           # IRAF starfind routine to detect star

from pymuse.detection import detect_unresolved_sources

In [ ]:
_roundness   = 0.8
_sharpnesslo = 0.1
_sharpnesshi = 0.9

sources = detect_unresolved_sources(galaxy,
                                    'OIII5006',
                                    StarFinder=DAOStarFinder,
                                    threshold=8,
                                    roundlo=-_roundness,
                                    roundhi=_roundness,
                                    sharplo=_sharpnesslo,
                                    sharphi=_sharpnesshi,
                                    save=False)

#### Compare to Kreckel et al. 2017

As mentioned in the beginning, we compare the newly detected sources to those from Kreckel et al. (2017). 

**Requires**
 * `match_coordinates_sky` from `astropy.coordinates` to compare the two catalogues.
 * `Angle` from `astropy.coordinates` to set a maximum seperation in units of arcseconds.

In [ ]:
from astropy.coordinates import match_coordinates_sky # match sources against existing catalog
from astropy.coordinates import Angle                 # work with angles (e.g. 1°2′3″)

tolerance = '0.5s'
ID, angle, Quantity  = match_coordinates_sky(pn_bright['SkyCoord'],sources['SkyCoord'])
within_tolerance = len(angle[angle.__lt__(Angle(tolerance))])

print(f'{within_tolerance} of {len(angle)} match within {tolerance}": {within_tolerance / len(angle)*100:.1f} %')
print(f'mean seperation is {angle.mean().to_string(u.arcsec,decimal=True)}"')

#### Plot detected sources

In [ ]:
from pymuse.plot.plot import plot_sky_with_detected_stars

In [ ]:
position = np.transpose((sources['x'], sources['y']))
positions_kk = np.transpose(pn_bright['SkyCoord'].to_pixel(wcs=galaxy.wcs))
positions = (position,positions_kk)

save_file = Path.cwd() / '..' / 'reports' / 'figures' / f'{galaxy.name}_sky_sources_DAO.pdf'
plot_sky_with_detected_stars(data=galaxy.OIII5006_DAP,
                             wcs=galaxy.wcs,
                             positions=positions,
                             filename=save_file)

#### Cut out detected stars

In [ ]:
from pymuse.plot.plot import sample_cutouts

In [ ]:
save_file = Path.cwd() / '..' / 'reports' / 'figures' / f'{galaxy.name}_stars.pdf'

stars = sample_cutouts(galaxy.OIII5006_DAP,sources,galaxy.wcs,nrows=4,ncols=4)

### Using SExtractor

there is no Python implementation of SExtractor. Instead we run it from the command line

```
sextractor file.fits -c default.sex
```

this will produce a file `test.cat` which contains the position of the sources. We read this table and calculate the sky position wiht astropy

In [ ]:
file = Path.cwd() / '..' / 'data' / 'interim' / 'NGC628.cat'

table = ascii.read(file)
table['SkyCoord'] = SkyCoord.from_pixel(table['X_IMAGE'],table['Y_IMAGE'],NGC628.wcs)

print(f'{len(table)} sources found')

In [ ]:
sources = Table()
sources['x'] = table['X_IMAGE']
sources['y'] = table['Y_IMAGE']
sources['SkyCoord'] = table['SkyCoord']
sources['fwhm'] = 0.8
NGC628.peaks_tbl = sources

#### Match with known sources

In [ ]:
ID, angle, Quantity  = match_coordinates_sky(pn_bright['SkyCoord'],table['SkyCoord'])
within_1_arcsec = len(angle[angle.__lt__(Angle("0.5s"))])

print(f'{within_1_arcsec} of {len(angle)} match within 0.5": {within_1_arcsec / len(angle)*100:.1f} %')
print(f'mean seperation is {angle.mean().to_string(u.arcsec,decimal=True)}"')
#print(f'mean angle: {angle.mean():.2f}')

#### Plot detected sources

this requires the previously loaded `plot_sources` from `pymuse.plot`

In [ ]:
file = Path.cwd() / '..' / 'reports' / 'figures' / f'{NGC628.name}_sources_sextractor.pdf'

position = np.transpose((sources['x'], sources['y']))
references = np.transpose(pn_bright['SkyCoord'].to_pixel(wcs=NGC628.wcs))
positions = (position,references)

sky_with_detected_stars(data=NGC628.OIII5006_old,wcs=NGC628.wcs,positions=positions,filename=file)

## Completeness limit

In [ ]:
from pymuse.detection import completeness_limit

In [ ]:
mock_sources = completeness_limit(
                   galaxy,
                   'OIII5006',
                   DAOStarFinder,
                   threshold=8,
                   iterations=1,
                   roundlo=-_roundness,
                   roundhi=_roundness,
                   sharplo=_sharpnesslo,
                   sharphi=_sharpnesshi
                    )

## Flux measurement

In the previous step we detected potential PN candidates by their [OIII] emission. This means we know their position but lack exact flux measurments. In this section we measure the flux of the identified objects in different emission lines that are used in later steps. 

### Growth curve analysis



#### Gaussian

A shape that is commonly used for the PSF is that of a 2D gaussian (we assume  variance of $\sigma_x^2 = \sigma_y^2 = \sigma^2$). If we center the peak at the origin the PSF is described by
$$
f(x,y) = A \exp\left(-\frac{x^2+y^2}{2\sigma^2}\right)
$$
with some amplitude $A$. We can rewrite this in polar coordinates as 
$$
f(r,\phi) = A \exp\left(-\frac{r^2}{2\sigma^2}\right)
$$
The light inside an aperture of radius $P(R)$ is given by the integral
$$
P(R) = \int_0^{2\pi} \int_0^R f(r,\phi) \mathrm{d} \phi r \mathrm{d} r = 2\pi \sigma^2 A \left(1-\exp \left(-\frac{R^2}{2\sigma^2}\right) \right) 
$$
We are interested in the ratio $p(R) = P(R) / P(\infty)$. If we use the relation between the standard deviation and the $\mathrm{FWHM}$ of a Gaussian $\sigma = \frac{\mathrm{FWHM}}{2\sqrt{2\ln2}}$, we can write
$$
\begin{align}p(R) = 1-\exp\left(- \frac{4 \ln 2 \cdot R^2}{\mathrm{fwhm}^2} \right)\end{align}
$$


#### Moffat

The measured FWHM are systematically larger than the reported values. A possible cause is that the shape of the PSF is not a perfect Gaussian, but rather described by a [Moffat](https://en.wikipedia.org/wiki/Moffat_distribution). This distribution is larger towards the wings and fitting a Gaussian to such a shape should result in a larger FWHM

$$
f(R;\alpha,\gamma) = A \left[1 + \left(\frac{R}{\gamma}\right)^2 \right]^{- \alpha}
$$

**Note**: this nomenclature follows `astropy` and contradicts the commonly used scheme which uses $\gamma=\alpha$ and $\alpha=\beta$.

The Full Width Half Maximum of this function is given by
$$
\mathrm{FWHM} = 2\gamma \sqrt{2^{1/\alpha}-1}
$$

like we did for the Gaussian we can calculate the amount of flux within a radius R as 

$$
\begin{align}
P(R) = \int_0^{2\pi} \int_0^R f(r,\phi) \mathrm{d} \phi r \mathrm{d} r = 2\pi \int_0^R A \left[1 + \left(\frac{r}{\gamma}\right)^2 \right]^{- \alpha} r \mathrm{d} r
\end{align}
$$

to solve this we substitute $u=1+\left(\frac{r}{\gamma} \right)^2 $ with $\frac{\mathrm{d} u}{\mathrm{d} r} = \frac{2r}{\gamma^2}$. 

$$
\begin{align}
P(R) = A \frac{\gamma^2}{2(1-\alpha)} \left[1 + \left( \frac{R}{\gamma} \right)^2 \right]^{1-\alpha}- A\frac{\gamma^2}{2(1-\alpha)}
\end{align}
$$


again we are interested in the ratio $p(R) = P(R) / P(\infty)$. If we assume that $\alpha>1$, the first term will be $0$ for $R\rightarrow \infty$ and so we end up with 

$$
p(r) = \left[ 1+\left( \frac{R}{\gamma}\right)^2\right]^{1-\alpha} - 1
$$

In [ ]:
from pymuse.photometry import light_in_gaussian, light_in_moffat, fwhm_moffat

alpha = 4
gamma = 10
fwhm = fwhm_moffat(alpha,gamma)
print(f'alpha={alpha:.2f}, gamma={gamma:.2f}, fwhm={fwhm:.2f}')

d = np.arange(0,20,0.2)
g = light_in_gaussian(d,fwhm)
m = light_in_moffat(d,alpha,gamma)
plt.plot(d/fwhm,100*g,label='Gaussian')
plt.plot(d/fwhm,100*m,label='Moffat')

plt.legend()
plt.xlabel('diameter in fwhm')
plt.ylabel('light in aperture in %')
plt.grid()

#### Create an ideal Gaussian/Moffat source and measure growth curve

In [ ]:
from astropy.modeling import models, fitting 
from astropy.nddata import Cutout2D
from astropy.stats import gaussian_sigma_to_fwhm, gaussian_fwhm_to_sigma

from pymuse.photometry import growth_curve
from pymuse.auxiliary import fwhm_moffat

In [ ]:
size=64
fwhm = 10
bkg = 0.5
print(f'fwhm={fwhm}')

std =  fwhm * gaussian_fwhm_to_sigma
gaussian = models.Gaussian2D(x_mean=size/2,y_mean=size/2,x_stddev=std,y_stddev=std)
img = gaussian(*np.indices((size,size))) + np.random.uniform(0,bkg,(size,size))
plt.imshow(img, origin='lower')
plt.show()


In [ ]:
fwhm_fit = growth_curve(img,size/2,size/2,model='gaussian',plot=True)[0]
print(f'fwhm={fwhm}, measured={fwhm_fit:.2f}')

In [ ]:
size=64
alpha = 4.765
gamma = 6/(2*np.sqrt(2**(1/4.76)-1))
bkg = 0.01

fwhm = 2*gamma * np.sqrt(2**(1/alpha)-1)
print(f'alpha={alpha:.2f}, gamma={gamma:.2f}, fwhm={fwhm:.2f}')

std = 4 * gaussian_fwhm_to_sigma
moffat = models.Moffat2D(x_0=size/2,y_0=size/2,alpha=alpha,gamma=gamma)

img = moffat(*np.indices((size,size))) + np.random.uniform(0,bkg,(size,size))
plt.imshow(img, origin='lower')
plt.show()

In [ ]:
fit = growth_curve(img,size/2,size/2,model='gaussian',plot=True,length=10)
fwhm_fit = fit[0]
#fwhm_fit = fwhm_moffat(*fit)

print(f'fwhm={fwhm:.2f}, fit={fwhm_fit:.2f}')

#### Apply to real data

We saw that we can predict the aperture size dependence of the flux for mock sources. Now we pick real objects and try to do the same. 

Stars should have much larger stellar velocities. We use this to identify potential foreground stars in our source catalogue. We cannot use the peaks of the velocity maps as they seem to be displaced from the center of the star.

In [ ]:
from scipy.signal import convolve

from astropy.nddata import Cutout2D
from astropy.stats import gaussian_fwhm_to_sigma

from pymuse.plot import single_cutout
from pymuse.photometry import growth_curve, correct_PSF, fwhm_moffat

we find stars due to their high velocity dispersion

In [ ]:
# define kernel for smoothing
smoothing_length = 10 
kernel = np.ones((smoothing_length,smoothing_length))

# find stars by their large velocity dispersion
star_mask = np.zeros(NGC628.V_STARS.shape,dtype='f8')
star_mask[np.abs(NGC628.V_STARS)>200] = 1

# smooth ouput with convolution
star_mask = convolve(star_mask,kernel,mode='same')
star_mask[star_mask>0.1] = 1
star_mask[star_mask<0.1] = 0
star_mask[np.isnan(NGC628.V_STARS)] = np.nan

In [ ]:
stars = detect_unresolved_sources(NGC628,
                                  'whitelight',
                                   StarFinder=DAOStarFinder,
                                   threshold=5,
                                   oversize_PSF = 1.,
                                   save=False)
    
stars = stars[star_mask[stars['y'].astype(int),stars['x'].astype(int)]==1]

In [ ]:
stars

In [ ]:
i = 0
size = 20
x,y,fwhm = stars[i][['x','y','fwhm']]
single_cutout(NGC628,'whitelight',x,y,size=size)
print(fwhm)

In [ ]:
data = NGC628.HA6562

aperture = 25
fit = growth_curve(data,x,y,model='moffat',rmax=15,plot=True,length=10)
#fwhm_fit=fit[0]
fwhm_fit = fwhm_moffat(*fit)

radius = np.arange(0,10,0.5)
plt.plot(radius,light_in_gaussian(radius,fwhm),label='gaussian',ls='--',color='tab:red')
plt.legend()

print(f'reported={fwhm:.2f}, measured={fwhm_fit:.2f}, ratio={fwhm_fit/fwhm:.2f}')

we have 6 objects classified as stars in our field of view. For each of them we do a growth curve analysis

In [ ]:
size = 20
data = NGC628.whitelight

fig,ax = plt.subplots(figsize=(8,5))

for i in range(len(stars)):
    x,y,fwhm = stars[i][['x','y','fwhm']]

    fit = growth_curve(data,x,y,model='moffat',rmax=30,plot=True,length=10)
    fwhm_fit = fwhm_moffat(*fit)
    print(f'alpha={fit[0]:.2f}, gamma={fit[1]:.2f}, reported={fwhm:.2f}, measured={fwhm_fit:.2f}, ratio={fwhm_fit/fwhm:.2f}')

#### Fit 2D Function

So far we didn't fit the PSF shape directly but took a slight detour with the light inside an aperture. Here we try to fit the 2D functions to the observed data

In [ ]:
size = 16

#sub = sources[sources['peak']>500]
#x,y,fwhm = sub[8][['x','y','fwhm']]

for line, cmap in zip(['whitelight','OIII5006','HA6562'],[plt.cm.viridis,plt.cm.Blues_r,plt.cm.Reds_r]):
    # defien the size of the cutout region
    star = Cutout2D(getattr(NGC628,line), (x,y), u.Quantity((size, size), u.pixel))

    fitter = fitting.LevMarLSQFitter()
    data = star.data
    fig ,(ax1,ax2,ax3,ax4) = plt.subplots(1,4,figsize=(12,3))
    #cmap = plt.cm.Blues

    ax1.imshow(data,origin='lower',cmap=cmap)
    ax1.set_title(f'image {line}')

    std = fwhm * gaussian_fwhm_to_sigma
    gaussian_theory = models.Gaussian2D(x_mean=size/2,y_mean=size/2,x_stddev=std,y_stddev=std)
    ax2.imshow(gaussian_theory(*np.indices(data.shape)), origin='lower',cmap=cmap)
    ax2.set_title('Gaussian reported')
    
    model = models.Gaussian2D()
    gaussian = fitter(model,*np.indices(data.shape),data,maxiter=1000)
    ax3.imshow(gaussian(*np.indices(data.shape)), origin='lower',cmap=cmap)
    ax3.set_title('Gaussian fit')

    model = models.Moffat2D(alpha=4.765,fixed={'alpha':True}) 
    moffat = fitter(model,*np.indices(data.shape),data,maxiter=2000)
    fwhm_moffat = 2*moffat.gamma.value * np.sqrt(2**(1/moffat.alpha.value)-1)
    ax4.imshow(moffat(*np.indices(data.shape)), origin='lower',cmap=cmap)
    ax4.set_title('Moffat fit')
    
    plt.tight_layout()
    xstd = gaussian.x_stddev * gaussian_sigma_to_fwhm
    ystd = gaussian.y_stddev * gaussian_sigma_to_fwhm

    print(f'{line}: reported: {fwhm:.3f}, gaussian={xstd:.3f} ,{ystd:.3f}, moffat={fwhm_moffat:.3f}')


In [ ]:
def fast_photometry(data,x,y,r,r_in,r_out):
    '''
    performe aperture photometry with backround subtraction for one source
    
    Parameters
    ----------
    data : ndarray
        image data
        
    x : float
        x cooridinate of the source
    
    y : float
        y cooridinate of the source
        
    r : float
        radius of the main aperture
    
    r_in : float
        inner radius of the annulus that is used for the background        
    
    r_out : float
        outer radius of the annulus that is used for the background
    
    '''
    
    aperture = CircularAperture((x,y), r=r)
    annulus_aperture = CircularAnnulus((x,y), r_in=r_in, r_out=r_out)
    mask = annulus_aperture.to_mask(method='center')
    annulus_data = mask.multiply(data)
    annulus_data_1d = annulus_data[mask.data > 0]
    _, bkg_median, _ = sigma_clipped_stats(annulus_data_1d[~np.isnan(annulus_data_1d)])
    phot = aperture_photometry(data,aperture)
    
    return phot['aperture_sum'][0]-aperture.area*bkg_median

def aperture_correction(data,positions,r_aperture):
    
    fluxes = [fast_photometry(data,x,y,r,r_in,r_out) for x,y in positions]
    
    apertures = CircularAperture(positions,r_aperture)
    

In [ ]:
from scipy.spatial import cKDTree
from pymuse.plot import single_cutout

positions = np.transpose([sources['x'],sources['y']])

tree = cKDTree(positions)
dists = tree.query(positions, 2)
nn_dist = dists[0][:, 1]

sub = sources[(nn_dist>16)]

### Aperture Photometry

we use the positions of the previously detected sources to measure the flux of different lines

https://photutils.readthedocs.io/en/stable/aperture.html

the values in the pixels are in units of $10^{-20} \ \mathrm{erg}  \ \mathrm{cm}^{-2} \ \mathrm{s}^{-1} / \mathrm{spaxel}$. For the [OIII] line, this flux is then converted to an apparent magnitude
$$
m_{[\mathrm{O\ III}]} = -2.5 \cdot \log F_{[\mathrm{O\ III}]} - 13.74
$$

where $F_{[\mathrm{O\ III}]}$ is given in $\mathrm{erg}  \ \mathrm{cm}^{-2} \ \mathrm{s}^{-1}$. Error propagation gives the error of the magnitude as

$$
\Delta m_{[\mathrm{O\ III}]} = \sqrt{\left(\frac{-2.5 \cdot \Delta F_{[\mathrm{O\ III}]}}{\ln 10 \cdot F_{[\mathrm{O\ III}]}}\right)^2 }
$$

We only correct for extinction in the milky way. therefor we use the extinction function from Cardelli, Clayton & Mathis (1989) with $A_V = 0.2$ and $R_V=3.1$. The extinction is calculated with the following package

https://extinction.readthedocs.io/en/latest/

(Note: the DAP products are already extinction corrected).

**Requires**
 * `extinction` a python package to account for the extinction in the Milky Way.
 * `measure_flux` from `pymuse.photometry`
 
**Returns**
 * `flux` a Table with the measured line fluxes.

In [ ]:
from astropy.coordinates import match_coordinates_sky # match sources against existing catalog
from astropy.coordinates import Angle                 # work with angles (e.g. 1°2′3″)
from astropy.coordinates import SkyCoord

from extinction import ccm89     # calculate extinction Cardelli et al. (1989)

from pymuse.photometry import measure_flux

In [ ]:
alpha = galaxy.alpha

flux = measure_flux(galaxy,sources,alpha,aperture_size=2.,background='local')

#calculate astronomical coordinates for comparison
flux['SkyCoord'] = SkyCoord.from_pixel(flux['x'],flux['y'],galaxy.wcs)

# calculate magnitudes from measured fluxes
flux['mOIII'] = -2.5*np.log10(flux['OIII5006']*1e-20) - 13.74
flux['dmOIII'] = np.abs( 2.5/np.log(10) * flux['OIII5006_err'] / flux['OIII5006'] )

# correct for milky way extinction
extinction = ccm89(wave=np.array([5007.]),a_v=0.2,r_v=3.1,unit='aa')[0]
flux['mOIII'] -= extinction

#### Compare to Kreckel et al. 2017

In [ ]:
from astropy.coordinates import match_coordinates_sky # match sources against existing catalog
from astropy.coordinates import Angle                 # work with angles (e.g. 1°2′3″)
from astropy.table import hstack

In [ ]:
ID, angle, Quantity  = match_coordinates_sky(pn_bright['SkyCoord'],flux['SkyCoord'])

# for each object from Kreckel et al. 2017, we search for the nearest source
# and copy our measured quantities to compare the two
pn_bright['mOIII_measured']  = flux[ID]['mOIII']
pn_bright['dmOIII_measured'] = flux[ID]['dmOIII']
pn_bright['sep'] = angle

fig,ax = plt.subplots(figsize=(7,7))

# we only use sources when their position agrees within this tolerance
tolerance = '0.5"'

# calculate the difference in magnitude for those objects
dif = np.mean(np.abs(pn_bright[angle<Angle(tolerance)]['mOIII'] - pn_bright[angle<Angle(tolerance)]['mOIII_measured']))

print(f'{len(pn_bright[angle<Angle(tolerance)])} PN match within {tolerance}')
print(f'the mean deviation is {dif:.3f} dex')

ax.errorbar(pn_bright[angle<Angle(tolerance)]['mOIII'],
            pn_bright[angle<Angle(tolerance)]['mOIII_measured'],
            yerr=pn_bright[angle<Angle(tolerance)]['dmOIII_measured'],
            fmt='o')

ax.scatter(pn_bright[angle>Angle(tolerance)]['mOIII'],pn_bright[angle>Angle(tolerance)]['mOIII'],color='tab:orange')

ax.plot([25.5,27.5],[25.5,27.5],color='black',lw=0.4)
ax.plot([25.5,27.5],[25.,27.],color='black',lw=0.2,ls='--')
ax.plot([25.5,27.5],[26.,28.],color='black',lw=0.2,ls='--')
ax.set_xlabel(r'$\mathrm{m}_{[\mathrm{OIII}]}$ Kreckel et al. 2017',fontsize=16)
ax.set_ylabel(r'$\mathrm{m}_{[\mathrm{OIII}]}$ this work',fontsize=16)

plt.show()

In [ ]:
from pymuse.plot.plot import single_cutout
x,y=pn_kreckel[np.argmax(pn_kreckel['mOIII'])]['SkyCoord'].to_pixel(wcs=galaxy.wcs)

single_cutout(galaxy,'OIII5006_DAP',x,y)

while the OIII magnitudes agree fairly well, we see a huge discrepancy in the H$\alpha$ fluxes.

In [ ]:
table = hstack([pn_bright,flux[ID]])

for col in ['OIII5006','HA6562','SII6716','NII6583']:
    table[col][table[col]<0] = table[f'{col}_err'][table[col]<0] 
    
table['OIII/Ha_measured'] = table['OIII5006'] / table['HA6562']
table['Ha/SII_measured'] = table['HA6562'] / table['SII6716']
table['Ha/NII_measured'] = table['HA6562'] / table['NII6583']

    
# print all relevant columns
table[table['sep']<Angle('0.5s')][['mOIII_1','mOIII_2','OIII/Ha','OIII/Ha_measured','Ha/SII','Ha/SII_measured','Ha/NII','Ha/NII_measured']]

In [ ]:
plt.scatter(tbl['HA6562_apbkg'],tbl['HA6562']/tbl['HA6562_err'])
plt.xlabel(r'$\mathrm{H}\alpha$ Background',fontsize=16)
plt.ylabel(r'$\mathrm{H}\alpha / \mathrm{H}\alpha_\mathrm{err}$',fontsize=16)
plt.show()

In [ ]:
plt.scatter(tbl['HA6562_apbkg'],tbl['HA6562']/tbl['HA6562_apbkg'])
plt.xlabel(r'$\mathrm{H}\alpha$ Background',fontsize=16)
plt.ylabel(r'$\mathrm{H}\alpha / \mathrm{H}\alpha_\mathrm{err}$',fontsize=16)
plt.show()

In [ ]:
np.sum(tbl['HA6562']/tbl['HA6562_err']<3)

## Emission line diagnostics

We built a catalgoue of possible planetary nebula and measuerd different emission lines. However this catalogue still contains objects that are similar to PN like HII regions or supernova remenants (SNR). In this next step we use emission line diagnostics to eliminate those contanimations. The distance modulus $\mu$ is defined as the difference between the apparent and the absolute magnitude. By definition of the absolute magnitude, this relates to the distance $d$ in parsec as 
$$
\begin{align}
\mu = m - M \\
d = 10^{1+\frac{\mu}{5}}
\end{align}
$$

 1. filter out HII regions
    $$
     4 > \log_{10} \frac{[\mathrm{OIII}]}{\mathrm{H}\alpha +[\mathrm{NII}]} > -0.37 M_{[\mathrm{OIII}]} - 1.16
    $$
 2. filter out SNR
    $$
     \mathrm{H}\alpha / [\mathrm{SII}] < 2.5
    $$
    
 3. estimate completness limit and remove fainter sources
    

In [ ]:
from pymuse.analyse import emission_line_diagnostics
    
tbl = emission_line_diagnostics(flux,29.91,27.5)

In [ ]:
filename = basedir / 'reports' / 'catalogues' / f'pn_candidates_{NGC628.name}.txt'
with open(filename,'w',newline='\n') as f:
    tbl['RaDec'] = tbl['SkyCoord'].to_string(style='hmsdms',precision=2)
    for col in tbl.colnames:
        if col not in ['id','RaDec','type']:
            tbl[col].info.format = '%.3f' 
    ascii.write(tbl[['id','type','x','y','RaDec','OIII5006','OIII5006_err','mOIII','dmOIII','HA6562','HA6562_err',
                          'NII6583','NII6583_err','SII6716','SII6716_err']][tbl['type']!='NaN'],
                f,format='fixed_width',delimiter='\t',overwrite=True)

### Compare velocity dispersion of PN to HII-regions and SNR

In [ ]:
completeness = 27.5
m = np.nanmean(tbl[tbl['mOIII']<completeness]['OIII5006_SIGMA'])
print(f'all: v_sig = {m}')

for t in ['PN','HII','SNR']:
    m = np.nanmean(tbl[(tbl['type']==t) & (tbl['mOIII']<completeness)]['OIII5006_SIGMA'])
    std = np.nanstd(tbl[(tbl['type']==t) & (tbl['mOIII']<completeness)]['OIII5006_SIGMA'])

    print(f'{t}: v_sig = {m:.2f} +- {std:.2f}')

### Visualize the result of the classification

In [ ]:
from pymuse.plot.pnlf import plot_emission_line_ratio
plot_emission_line_ratio(tbl,mu = 29.91)

In [ ]:
from photutils import CircularAperture
from astropy.visualization import simple_norm

# ====== define input parameters =============================
galaxy = NGC628
labels=['SII6716','HA6562','OIII5006']
wcs=NGC628.wcs
# ============================================================

table = tbl

fig, (ax1,ax2) = plt.subplots(ncols=2,figsize=(20,10),subplot_kw={'projection':wcs})

norm = simple_norm(galaxy.HA6562,'linear',clip=False,max_percent=95)
ax1.imshow(galaxy.HA6562,norm=norm,cmap=plt.cm.Greens_r)

norm = simple_norm(galaxy.OIII5006_DAP,'linear',clip=False,max_percent=95)
ax2.imshow(galaxy.OIII5006_DAP,norm=norm,cmap=plt.cm.Blues_r)

for t,c in zip(['HII','SNR','PN'],['black','red','yellow']):
    
    sub = table[table['type']==t]
    positions = np.transpose([sub['x'],sub['y']])
    apertures = CircularAperture(positions, r=6)
    apertures.plot(color=c,lw=.5, alpha=1,ax=ax1)
    apertures.plot(color=c,lw=.5, alpha=1,ax=ax2)

ax1.set_title('HA6562')
ax2.set_title('OIII5006')

plt.savefig(basedir / 'reports' / 'figures' / 'NGC628_detections_classification.pdf')

### Compare classification to the results from Francesco Santoro

In [ ]:
from pymuse.detection import match_catalogues

In [ ]:
with fits.open(basedir / 'data' / 'external' / 'FS_cat_v01.fits') as hdul:
    cat_FS = Table(hdul[1].data)

Search for PNe that were classified by Francescos in my catalogue

In [ ]:
del tbl['SkyCoord']
PNe_candidates = cat_FS[(cat_FS['gal_name']=='NGC628') & (cat_FS['PNe_candidate']==1)]
idx, sep = match_catalogues(PNe_candidates[['cen_x','cen_y']],tbl[['x','y']])

max_sep = 2
print(f'sep < {max_sep} px: {sum(sep<max_sep)/len(sep)*100:.2f} %')
tbl[(idx)][sep<max_sep]

#### Compare the measured fluxes

In [ ]:
tmp = hstack([PNe_candidates,tbl[(idx)][sep<max_sep]])

tmp[['OIII5006_FLUX', 'OIII5006', 'HA6562_FLUX', 'HA6562', 'NII6583_FLUX','NII6583', 'SII6716_FLUX' ,'SII6716']]

Search for HII regions that were classified by me in Francescos catalogue

In [ ]:
HII_candidates = tbl[tbl['type'] == 'HII']
catalogue = cat_FS[(~np.isnan(cat_FS['cen_x'])) & (~np.isnan(cat_FS['cen_y'])) & (cat_FS['gal_name']=='NGC628')]
idx, sep = match_catalogues(HII_candidates[['x','y']],catalogue[['cen_x','cen_y']])

max_sep = 2
print(f'sep < 1 px: {sum(sep<1)/len(sep)*100:.2f} %')
catalogue[idx][sep<max_sep]

## Planetary nebula luminosity function

The absolute magnitude of PN is described by (this is an empirical relation)
$$ 
\begin{align}
N(M) &\propto e^{0.307 M} \left( 1- e^{3(M^*-M)} \right) \\
&\propto e^{0.307 (m-\mu)} \left( 1- e^{3(M^*-m+\mu)} \right) \\
&\propto e^{0.307 (m-\mu)} - e^{3M^*-2.693(m-\mu)} 
\end{align}
$$
To use this function in our Maximum Likelihood we need to normalize it. The indefinite integral is
$$
\begin{align}
\int N(m)\; \mathrm{d} m \propto \frac{e^{0.307(m-\mu)}}{0.307} + \frac{e^{3M^* - 2.693(m-\mu)}}{2.693}
\end{align}
$$
The luminosity function has a root when $M^* - m + \mu =0$. We use this for the lower bound normalization. For the upper bound we use the luminosity for which we are confindent to detect all sources (=completeness limit).

We can already use the normalized luminosity function for the maximum likelihood fitting. However we cannot really illustrate the result. To do this we need to introduce some binning. Then we can show the fit similar to a curve fit. Because we sum the PN in the bins, we don't plot the luminosity function but the integrated function. 

### With maximum likelihood

**Note**: the function which is used for the likelihood must be normalized


In [ ]:
def prior(mu):
    mu0 = 29.91
    std = 0.5
    
    return 1 / (std*np.sqrt(2*np.pi)) * np.exp(-(mu-mu0)**2 / (2*std**2))

In [ ]:
from pymuse.analyse import MaximumLikelihood, PNLF, pnlf

completeness = 29.5

criteria = ((tbl['type']=='PN'))
data = tbl[np.where(criteria & (tbl['mOIII']<completeness))]['mOIII']
err  = tbl[np.where(criteria & (tbl['mOIII']<completeness))]['dmOIII']
#data = data[data>27]
fitter = MaximumLikelihood(pnlf,
                           data,
                           prior=prior,
                           mhigh=completeness)

# a good guess would be mu_guess = min(data)-Mmax
mu = fitter([28])[0]

### Plot the fit

to plot the fit we need to bin the data

In [ ]:
from pymuse.plot.pnlf import plot_pnlf
filename = basedir / 'reports' / 'figures' / f'{galaxy.name}_PNLF'

plot_pnlf(tbl[criteria]['mOIII'],mu,completeness,binsize=0.25,mhigh=32)

In [ ]:
import scipy.optimize as op
from pymuse.analyse import F

def f(mu,m,mhigh,Mmax):
    mlow = Mmax + mu
    normalization =  (np.exp(0.307*mu)*(np.exp(0.307*mlow)-np.exp(0.307*mhigh)) + np.exp(2.693*mu)*(np.exp(3*Mmax-2.693*mhigh)-np.exp(3*Mmax-2.693*mlow))) / (F(mhigh,mu)-F(mlow,mu))
    like = 0.307*len(m)*mu+np.sum(3/(np.exp(-3*(Mmax-m+mu))-1))

    return normalization + like

op.newton(f,x0=29,args=(data,29,-4.47))


In [ ]:
def contaminate(tbl,percent):
    
    true  = np.where(tbl['type']=='PN')[0]
    false = np.where(tbl['type']=='SNR')[0]
    
    false = np.random.choice(false,int(len(true)*percent),replace=False)
    np.concatenate(true,false,out=true)

    return tbl[true]

contaminate(tbl,0.1)

### With data from Kreckel et al. 2017

In [ ]:
fitter = MaximumLikelihood(pnlf,pn_bright['mOIII'],mhigh=27)
mu = fitter([25])[0]
plot_pnlf(pn_kreckel['mOIII'],mu=29.91,completeness=27,binsize=0.4)

### With least square  fitting

 - to use a least square approach we need to bin the data. 
 + easier to implement

In [ ]:
from scipy.optimize import curve_fit

def pnlf(m,mu,N0):
    '''planetary nebula luminosity function for curve_fit
    
    N(m) ~ e^0.307(m-mu) * (1-e^3(Mmax-m+mu))
    
    Parameters
    ----------
    m : ndarray
        apparent magnitudes of the PNs
        
    mu : float
        distance modulus
        
    N0 : float
    '''
    
    m = np.atleast_1d(m)
    
    Mmax = -4.47
    completneness = 28
    normalization = -3.62866*np.exp(0.307*Mmax) + 3.25733*np.exp(0.307*completneness-0.307*mu) + 0.371333 * np.exp(3*Mmax - 2.693 * completneness + 2.693 * mu)
    
    out = N0*np.exp(0.307*(m-mu)) * (1-np.exp(3*(Mmax-m+mu))) / normalization
    out[m>completneness] = 0
    out[m<Mmax+mu] = 0
    
    return out

def fit_pnlf(table):
    
    #table = table[table['type']=='PN']
    
    binsize = 0.2

    guess = np.array([25,10])
    
    mlow = np.floor(np.min(table))
    mhigh = np.ceil(np.max(table))
    hist,bins  = np.histogram(table,np.arange(mlow,mhigh,binsize))
    
    
    fit,sig = curve_fit(pnlf, bins[1:]+binsize/2,hist , guess)
    mu, N0 = fit
    print(f'mu={mu:.3f}, N0={N0:.2f}')
    
    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(8,4))
    
    ax1.scatter(bins[:-1]+binsize/2,hist)
    ax1.plot(bins[:-1]+binsize/2,pnlf(bins[:-1]+0.1,mu=mu,N0=N0),c='tab:orange',ls='--')
    ax1.set_yscale('log')
    ax1.set_xlim([25,28])
    ax1.set_ylim([0.8,1.1*np.max(hist)])
    ax1.set_xlabel('$m_{[\mathrm{OIII}]}$')
    ax1.set_ylabel('$N$')
    
    ax2.plot(bins[1:]+binsize/2,np.cumsum(hist))
    ax2.plot(bins[1:]+binsize/2,np.cumsum(pnlf(bins[:-1]+binsize/2,mu=mu,N0=N0)),ls='--')
    ax2.set_xlim([mlow,mhigh])
    ax2.set_ylim([0,len(table)])
    ax2.set_xlabel('$m_{[\mathrm{OIII}]}$')
    ax2.set_ylabel('Cumulative N')
    
fit_pnlf(tbl[tbl['type']=='PN']['mOIII'])

### Distance in parsec

the measured distances are in the form of the distance modulus $\mu = m-M$ which is the difference between apparent and absolute magnitude. By defintion of the absolte magnitude, we can convert this number into a distance in pc
$$
d = 10^{\frac{\mu}{5}+1} = 10 \cdot \exp\left( \ln 10 \frac{\mu}{5} \right) \\
\delta d = \frac{\ln 10}{5} 10 \exp\left( \ln 10 \frac{\mu}{5} \right) \delta \mu = 0.2 \ln 10 \; d \; \delta \mu
$$

In [ ]:
def distance_modulus_to_parsec(mu,mu_err=np.array([])):
    
    d = 10 * np.exp(np.log(10)*mu/5)
    if len(mu_err) > 0:
        d_err = 0.2 * np.log(10) * d * mu_err
    print(f'd = ({d/1e6:.2f} + {d_err[0]/1e6:.2f} - {d_err[1]/1e6:.2f}) Mpc')
    
    return d, d_err

d,d_err = distance_modulus_to_parsec(30.033,np.array([0.014,0.015]))

## Playground

In [ ]:
with fits.open(data_raw / 'NGC628' / 'NGC628_star_mask.fits') as hdul:
    star_mask =hdul[0].data

In [ ]:
from pymuse.plot import create_RGB

# ====== define input parameters =============================
rgb = create_RGB(NGC628.SII6716,NGC628.HA6562,NGC628.OIII5006)
labels=['SII6716','HA6562','OIII5006']
wcs=NGC628.wcs
# ============================================================

# create an empty figure with correct projection
fig, ax = plt.subplots(figsize=(20,20),subplot_kw={'projection':wcs})

# plot the image
plt.imshow(rgb,origin='lower')

# create a legend
if labels:
    # first we create a legend with three invisible handles
    handles = 3*[mpl.patches.Rectangle((0, 0), 0, 0, alpha=0.0)]
    leg = ax.legend(handles,labels, frameon=False,handlelength=0,prop={'size': 16})

    # next we set the color of the three labels
    for color,text in zip(['red','green','blue'],leg.get_texts()):
        text.set_color(color)

plt.savefig(basedir / 'reports' / 'figures' / f'{NGC628.name}_rgb.pdf')
plt.show()